<a href="https://colab.research.google.com/github/sabaagizew/USGS_LIDAR_Challenge/blob/main/USGS_LIDAR_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pdal
import json
import laspy
import geopandas as gpd
import numpy as np
from shapely.geometry import box, Point, Polygon

In [ ]:
class FetchAndLoad():
    
    def __init__(self, polygon:Polygon, epsg:int=4326, state='IA_FullState'):
        self.polygon = polygon

        self.state = state
        self.epsg = epsg
        self.bounds = None
        self.crs_polygon = None
        
        self.pipe_path = './pipeline.json'
        self.las_path = self.state+'.las'
        self.tif_path = self.state+'.tif'
        self.api_path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"+self.state+"/ept.json"
        
        self.pipeline = None
        
        self.las_file = None
        self.points = None
        self.elevation = None
        self.geo_df = None
        
        
    def read_json(self):
  
      try:
        file_path = self.pipe_path
        print("File Path : ", file_path)
        with open(file_path, 'r') as json_file:
          data = json.loads(json_file.read())
        return data

      except:
        print('File Not found')
        return None
    
    def get_bounds_and_ploygon(self):
      
        polygon_df = gpd.GeoDataFrame([self.polygon], columns=['geometry'])
        
        polygon_df.set_crs(epsg=self.epsg, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(
            epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds
        
        polygon_input = 'POLYGON(('

        xcord, ycord = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcord), list(ycord)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'
        
        
        self.bounds = f"({[minx, maxx]},{[miny,maxy]})"
        self.crs_polygon = polygon_input
        
        
    def prepare_pipe(self):
        self.get_bounds_and_ploygon()
        data = self.read_json()
        data['pipeline'][0]['bounds'] = self.bounds
        data['pipeline'][0]['filename'] = self.api_path
        
        data['pipeline'][1]['polygon'] = self.crs_polygon
        
        data['pipeline'][4]['out_srs'] = f'EPSG:{self.epsg}'

        data['pipeline'][7]['filename'] = self.las_path
        data['pipeline'][8]['filename'] = self.tif_path


        print("data LInk : " , data['pipeline'][0]['filename'])
        self.pipeline = data
        
    
    def run_pipe(self):
        
        print("Run pipe ...")
        result = self.prepare_pipe()
        pdal_pipe = pdal.Pipeline(json.dumps(self.pipeline))
        pdal_result = pdal_pipe.execute()
        print("Fetching Completed!")
        
        
    def read_laz(self):
        try:
            print("Reading Las File from :", self.las_path)
            las = laspy.read(self.las_path)
            self.las_file = las
            return las
        
        except FileNotFoundError:
            print("Log: File Not found")
            print("Please use the function run_pipe before this funciton")
            
    
    def generate_points_elevation(self):
        print("Generating Points from las File ...")
        points = [Point(x, y) for x,y in zip(self.las_file.x, self.las_file.y)]
        elevation = np.array(self.las_file.z)
        
        self.points, self.elevation = points, elevation
        print("Finished Generating Points!")
        
        return points, elevation

In [ ]:
#inital inputs
MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.747334, 41.921429]

# MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.756055, 41.918115]

polygon = Polygon(((MINX, MINY), (MINX, MAXY),
                   (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))



check = FetchAndLoad(polygon)
check.run_pipe()

In [ ]:
#Generating Geopandas DF from las file
check.create_geopandasdf()
new_result =  check.get_geopandas_df()

In [ ]:
new_result.head()

In [ ]:
## Todo, Add Year and return the Data frame.

## Save the data frame if possible.
import matplotlib.pyplot as plt


fig, ax = plt.subplots(1, 1, figsize=(12, 10))

new_result.plot(column='elevation', ax=ax, legend=True)
plt.show()

In [ ]:
import numpy as np
import laspy as lp

point_cloud=lp.read("IA_FullState.las")

In [ ]:
points = np.vstack((point_cloud.x, point_cloud.y, point_cloud.z)).transpose()
colors = np.vstack((point_cloud.red, point_cloud.green, point_cloud.blue)).transpose()

In [ ]:
factor=160
decimated_points_random = points[::factor]

In [ ]:
voxel_size=6
nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)

In [ ]:
non_empty_voxel_keys, inverse, nb_pts_per_voxel = np.unique(((points - np.min(points, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
idx_pts_vox_sorted=np.argsort(inverse)

In [ ]:
voxel_grid={}
grid_barycenter,grid_candidate_center=[],[]
last_seen=0

In [ ]:
for idx,vox in enumerate(non_empty_voxel_keys):
    voxel_grid[tuple(vox)]= points[idx_pts_vox_sorted[
    last_seen:last_seen+nb_pts_per_voxel[idx]]]
    grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))
    grid_candidate_center.append(
       voxel_grid[tuple(vox)][np.linalg.norm(voxel_grid[tuple(vox)] -
       np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()])
    last_seen+=nb_pts_per_voxel[idx]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
decimated_colors = colors[::factor]
ax = plt.axes(projection='3d')
ax.scatter(decimated_points_random[:,0], decimated_points_random[:,1], decimated_points_random[:,2], c = decimated_colors/65535, s=0.01)
plt.show()